### Funcion de recompensa:

In [1]:
import numpy as np
def reward_function(params):
    
    def gaussian(x, mu, sig):
        return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
    
    def deltaDeg(direccionAuto, puntoInicial, puntoFinal):
        """ Esta funcion calcula la diferencia en grados (°Deg) entre la orientacion actual
            del vehiculo y la orientación de un vector abitrario, definido por dos puntos."""
        
        vector = [puntoFinal[0] - puntoInicial[0], puntoFinal[1] - puntoInicial[1]] #Vector que une la posicion del 
                                                                                              #auto con el punto arbitrario.
        degRumboVector = np.degrees(np.arctan2(vector[1],vector[0])) #Rumbo del vector respecto de la horizontal.
        deltaRumbo = np.absolute(degRumboVector - direccionAuto) #Diferencia entre el rumbo del vector y la direccion actual del auto.
        return float(deltaRumbo)
    
    reward = 1e-3
    
    #Defino mis variables/datos de entrada:
    #posicionActual = [params['x'],params['y']] #Posicion actual del vehiculo en el mundo/entorno que contiene la pista.
    waypoints = params['waypoints'] #Lista con las coordenadas x,y de los waypoints para la pista dada.
    closest_waypoints = params['closest_waypoints'] #Indices de los waypoints mas cercanos a la posicion actual del vehiculo.
                                                    #El primer elemento refiere al waypoint pasado más cercano y el segundo
                                                    #al proximo waypoint a cruzar.
    direccionActual = params['heading'] #Orientación del vehiculo respecto de la horizontal (°Deg, Easting)
    
    A = 2    #Peso para la recompensa por posicion dentro de la pista.
    B = 1    #Pero para la orientacion respecto de la orientacion de la pista.
    C = 0.25 #Castigo por salirse de la pista tempranamente.
    
    #-----------------------------------------------------------------------------------------------------------------------
    # POSICION: Recompensas otorgadas en base a la posicion del auto dentro de la pista:
    
    # Penalizacion por alejarse del centro de la pista:
    anchoPista = params['track_width']
    distanciaCentro = params['distance_from_center']
    if distanciaCentro > anchoPista*0.5:
        reward += A*gaussian(x=distanciaCentro, mu=0, sig = anchoPista/2*0.10)
    else:
        reward = 1e-3    
    
    #-----------------------------------------------------------------------------------------------------------------------
    # ORIENTACION: Recompensas otorgadas en base a la orientacion del auto respecto del rumbo de la pista.
    next_Waypoint = waypoints[closest_waypoints[1]]
    prev_Waypoint = waypoints[closest_waypoints[0]]
    
    # Se aproxima la orientación de la pista a partir del vector que unos los waypoints mas proximos, el último en pasar y 
    # el proximo por alcanzar
    deltaRumbo =  deltaDeg(direccionActual, prev_Waypoint, next_Waypoint)
    reward += B*gaussian(x = deltaRumbo, mu = 0, sig = 20)

    #-----------------------------------------------------------------------------------------------------------------------
    # VELOCIDAD:    
    # Penalización por doblar bruscamente a altas velocidades:  
    anguloGiro = np.abs(params['steering_angle'])
    velocidad = np.abs(params['speed'])
    if anguloGiro > 25 and velocidad > 2.0:
        reward *= C
        
    #-----------------------------------------------------------------------------------------------------------------------        
    # OTROS:
    # Descartar todo tipo de recompensa si el auto se salió tempranamente de la pista.
    if params["all_wheels_on_track"] == False and params["steps"] < 20:
        reward = 1e-3
        
    return float(reward)